In [1]:
import pandas as pd

from math import pi
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

pd.options.display.max_rows = 100

In [2]:
def run_cluster(data, num_clusters=5):
    # create kmeans model
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(data)

    # find centers and labels
    centers = kmeans.cluster_centers_
    labels = kmeans.labels_

    # DataFrame of centroids
    centroids_df = pd.DataFrame(centers, columns=data.columns).T

    # print items in clusters
    clusters = {}
    for i in range(len(labels)):
        clusters[players[i]] = labels[i]

    for i in range(len(centers)):
        out = [k for k,v in clusters.items() if v==i]

        print('-'*30)
        print(f'cluster index: {i}')
        for item in out:
            print(item)
        print('\n')

#     centroids_df.to_csv('final_clusters.csv')
    return clusters, centroids_df


In [3]:
data = pd.read_csv('Data/nba_data.csv')
players = data.player.to_list()


In [4]:
# nba_data = nba_data.set_index('rank')
nba_data = data.drop(['Unnamed: 0', 'player', 'pos', 'team_id', 'age', 'mp', 'fg', 'fga','fg3',
 'fg3a','fg2',
 'fg2a','ft',
 'fta','orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',], axis=1)


In [5]:
clusters, centroids_df = run_cluster(nba_data, 8)

------------------------------
cluster index: 0
Jaylen Adams
Grayson Allen
Justin Anderson
Dwayne Bacon
Mo Bamba
J.J. Barea
Jerryd Bayless
Dragan Bender
Antonio Blakeney
Jonah Bolden
Isaiah Briscoe
Ryan Broekhoff
Sterling Brown
Troy Brown
Jose Calderon
Jevon Carter
Gary Clark
Ian Clark
Allen Crabbe
Troy Daniels
Hamidou Diallo
Jacob Evans
Dante Exum
Raymond Felton
Channing Frye
Devonte' Graham
Treveon Graham
Jerian Grant
Solomon Hill
Aaron Holiday
Danuel House
Chandler Hutchison
Amir Johnson
Frank Kaminsky
Furkan Korkmaz
Luke Kornet
Jarell Martin
Frank Mason
Abdel Nader
Shabazz Napier
Raul Neto
Georges Niang
Dirk Nowitzki
Frank Ntilikina
David Nwaba
Semi Ojeleye
Elie Okobo
Tony Parker
Quincy Pondexter
Chasson Randle
Glenn Robinson
Jerome Robinson
Thabo Sefolosha
Ish Smith
Omari Spellman
Lance Thomas
Sindarius Thornwell
Moritz Wagner
Tyrone Wallace
Brad Wanamaker
D.J. Wilson
Guerschon Yabusele
Isaiah Canaan
Marquese Chriss
Sam Dekker
Matthew Dellavedova
Tyler Dorsey
Wesley Johnson
Brando

In [6]:
# Plots a radar chart.
def plot_radar(cat, values, title, fill_color='k', edge_color=None):
    N = len(cat)
    x_as = [n / float(N) * 2*pi for n in range(N)]

    # Because our chart will be circular we need to append a copy of the first 
    # value of each list at the end of each list with data
    values += values[:1]
    x_as += x_as[:1]

    # Set color of axes
    plt.rc('axes', linewidth=0.5, edgecolor="#888888")

    # Create polar plot
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, polar=True)
    ax.set_facecolor('#d3d3d3')

    # Set clockwise rotation. That is:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    # Set position of y-labels
    ax.set_rlabel_position(0)

    # Set color and linestyle of grid
    ax.xaxis.grid(True, color="#121212", linestyle='solid', linewidth=0.5)
    ax.yaxis.grid(True, color="#000000", linestyle='solid', linewidth=0.5)

    # Set number of radial axes and remove labels
    plt.xticks(x_as[:-1], [])

    # Set yticks
    plt.yticks([20, 40, 60, 80, 150], ["20", "40", "60", "80", "100"])

    # Plot data
    ax.plot(x_as, values, linewidth=0, linestyle='solid', zorder=3)

    # Fill area
    ax.fill(x_as, values, facecolor=fill_color, edgecolor=edge_color, alpha=0.4)

    # Set axes limits
    plt.ylim(0, max(values))

    # Draw ytick labels to make sure they fit properly
    for i in range(N):
        angle_rad = i / float(N) * 2 * pi

        if angle_rad == 0:
            ha, distance_ax = "center", 10
        elif 0 < angle_rad < pi:
            ha, distance_ax = "left", 1
        elif angle_rad == pi:
            ha, distance_ax = "center", 1
        else:
            ha, distance_ax = "right", 1

        ax.text(angle_rad, 100 + distance_ax, cat[i], size=10, horizontalalignment=ha, verticalalignment="center")

    ax.set_title(title, y=-0.05)

    # Show polar plot
    plt.show()


In [8]:
player_df = data[['player', 'team_id']]
player_df

test_df = pd.DataFrame([clusters]).T.reset_index()
test_df = test_df.rename(columns={'index':'player', 0:'cluster'})

test_df

player_type_df = player_df.merge(test_df, on='player')
player_type_df

player_type_df.sort_values(by='team_id')


,player,team_id,cluster
143,Daniel Hamilton,ATL,3
27,Kent Bazemore,ATL,4
33,DeAndre' Bembry,ATL,2
68,Vince Carter,ATL,2
76,John Collins,ATL,7
...,...,...,...
60,Thomas Bryant,WAS,7
58,Troy Brown,WAS,0
294,Tomas Satoransky,WAS,6
28,Bradley Beal,WAS,1
